In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import numpy as np
from skimage.segmentation import mark_boundaries

from htc.cpp import map_label_image
from htc.evaluation.evaluate_superpixels import EvaluateSuperpixelImage
from htc.models.image.DatasetImage import DatasetImage
from htc.settings_seg import settings_seg
from htc.tivita.DataPath import DataPath
from htc.utils.Config import Config

In [2]:
config = Config.load_config("default_rgb", "superpixel_classification")
config["trainer_kwargs/precision"] = 32
sample = DatasetImage([DataPath.from_image_name("P058#2020_05_13_17_31_55")], train=False, config=config)[0]
spxs = sample["spxs"].numpy()
rgb = sample["features"].numpy()
label_mapping = settings_seg.label_mapping
label_mapping.label_colors = settings_seg.label_colors_paper

eval_spxs = EvaluateSuperpixelImage().evaluate_cpp(sample)
seg = map_label_image(sample["labels"], label_mapping).numpy()
spx_seg = map_label_image(eval_spxs["predictions"], label_mapping).numpy()

In [3]:
color = mpl.colors.to_rgb(settings_seg.model_colors["superpixel_classification"])
mpl.image.imsave(
    settings_seg.paper_dir / "model_overview_spx.png",
    mark_boundaries(seg[:, :, :3], spxs, color=color, outline_color=color, mode="outer"),
    vmin=0,
    vmax=1,
)

In [4]:
seg_patch = np.zeros((480, 640), dtype="int64")
val = 0
for i in np.arange(640 // 64):
    for j in np.arange(480 // 64 + 1):
        seg_patch[j * 64 :, i * 64 : (i + 1) * 64] = val
        val += 1

color = mpl.colors.to_rgb(settings_seg.model_colors["patch_64"])
mpl.image.imsave(
    settings_seg.paper_dir / "model_overview_patch.png",
    mark_boundaries(seg[:, :, :3], seg_patch, color=color, outline_color=color, mode="outer"),
    vmin=0,
    vmax=1,
)

color = (0.6, 0.6, 0.6)
mpl.image.imsave(
    settings_seg.paper_dir / "model_overview_patch_rgb.png",
    mark_boundaries(rgb, seg_patch, color=color, outline_color=color, mode="outer"),
    vmin=0,
    vmax=1,
)

In [5]:
seg_pixel = np.zeros((480, 640), dtype="int64")
val = 0
for i in np.arange(0, 640, 6):
    for j in np.arange(0, 480, 6):
        seg_pixel[j : j + 6, i : i + 6] = val
        val += 1

color = mpl.colors.to_rgb(settings_seg.model_colors["pixel"])
mpl.image.imsave(
    settings_seg.paper_dir / "model_overview_pixel.png",
    mark_boundaries(seg[:, :, :3], seg_pixel, color=color, mode="inner"),
    vmin=0,
    vmax=1,
)

color = (0.6, 0.6, 0.6)
mpl.image.imsave(
    settings_seg.paper_dir / "model_overview_pixel_rgb.png",
    np.clip(mark_boundaries(rgb, seg_pixel, color=color, mode="subpixel"), 0, 1),
    vmin=0,
    vmax=1,
)

In [6]:
mpl.image.imsave(settings_seg.paper_dir / "model_overview_image.png", seg)